# Imports

In [ ]:
!pipenv install pyproj

In [ ]:
import requests
import pandas as pd
import time
import os
import json
from datetime import datetime, timedelta
import re
import httpx

# kvkdf en kvk nummers uit export van PowerBi

In [ ]:
# Lees de KvK-nummers uit de Hoofdvestigingen.xlsx en forceer de qNawKvk kolom als tekst
kvk_df = pd.read_excel('Hoofdvestigingen 10M.xlsx', dtype={'qNawKvk': str})

# Haal de KvK-nummers op uit de qNawKvk kolom
kvk_nummers = kvk_df['qNawKvk'].dropna().astype(str).tolist()  # Zorg ervoor dat het een lijst van strings is



# begin van api call naar zorgaanbiedersportaal

In [ ]:
# Basis URL voor de API
base_url = "https://zoeken.zorgaanbiedersportaal.nl/api/organisatie/"  # F-string placeholder

# Zorg ervoor dat de responses map bestaat
os.makedirs('./responses', exist_ok=True)



In [ ]:
# Bestand voor het opslaan van de tijd van de laatste call
last_call_time_file = './last_call_time.txt'

# Functie voor het ophalen van de tijd van de laatste call
def get_last_call_time():
    if os.path.exists(last_call_time_file):
        with open(last_call_time_file, 'r') as f:
            return datetime.fromisoformat(f.read().strip())
    return None

# Functie voor het opslaan van de tijd van de laatste call
def save_last_call_time():
    with open(last_call_time_file, 'w') as f:
        f.write(datetime.now().isoformat())

In [ ]:
# Functie voor het ophalen van data van de API
def get_data(kvknummer):
    url = f"{base_url}{kvknummer}"
    try:
        response = requests.get(url,timeout=2)
        # Check of de response succesvol is
        if response.status_code == 200:
            # Sla de response op in een bestand in de ./responses map
            with open(f'./responses/{kvknummer}.json', 'w', encoding='utf-8') as f:
                json.dump(response.json(), f, ensure_ascii=False, indent=4)
            print(f"Data voor KvK-nummer {kvknummer} opgeslagen.")
        else:
            print(f"Fout bij ophalen data voor KvK-nummer {kvknummer}. Statuscode: {response.status_code}")
            try:
                error_info = response.json()  # Probeer de foutmelding uit de JSON response te halen
                print(f"Foutmelding: {error_info.get('message', 'Geen specifieke foutmelding beschikbaar.')}")
            except json.JSONDecodeError:
                print("Geen geldige JSON ontvangen in de foutmelding.")
    except requests.exceptions.RequestException as e:
        print(f"Netwerkfout bij het ophalen van data voor KvK-nummer {kvknummer}: {str(e)}")

# Functie om de calls te rate-limiten en ervoor te zorgen dat er niet meer dan 50 calls per uur worden gemaakt
def rate_limit_calls():
    last_call_time = get_last_call_time()

    if last_call_time:
        # Bereken hoeveel tijd er is verstreken sinds de laatste call
        elapsed_time = datetime.now() - last_call_time
        if elapsed_time < timedelta(hours=1):
            # Als minder dan een uur is verstreken, wacht dan totdat een uur is verstreken
            wait_time = timedelta(hours=1) - elapsed_time
            print(f"Maximaal aantal calls per uur bereikt. Wachten voor {wait_time}.")
            time.sleep(wait_time.total_seconds())
    # Sla de tijd van de laatste call op na het wachten
    save_last_call_time()



# Haal het laatst verwerkte KvK-nummer op (indien aanwezig)
def get_last_processed_kvk():
    if os.path.exists('./last_processed_kvk.txt'):
        with open('./last_processed_kvk.txt', 'r') as f:
            return f.read().strip()
    return None

# Sla het laatst verwerkte KvK-nummer op
def save_last_processed_kvk(kvknummer):
    with open('./last_processed_kvk.txt', 'w') as f:
        f.write(kvknummer)


# Verwerk de data voor alle KvK-nummers door de bestanden in ./responses te laden
last_processed_kvk = get_last_processed_kvk()
start_index = kvk_nummers.index(last_processed_kvk) + 1 if last_processed_kvk else 0

counter =0
rate_limit_calls()

for kvknummer in kvk_nummers[start_index:]:
    counter+=1
    print(f"Count {counter}",f"Verwerken van data voor KvK-nummer {kvknummer}")
    if counter == 120:
        rate_limit_calls()  # Controleer en wacht indien nodig
        counter = 0
    get_data(kvknummer)
    save_last_processed_kvk(kvknummer)

# haal data uit response jsons zorgdata api op

In [ ]:
# Lege lijsten voor de data
sbi_data = []
vestigingen_data = []
overige_data = []
pand_data = []

# Functie om de data uit een JSON-bestand te laden en te verwerken
def process_data_from_file(kvknummer):
    try:
        with open(f'./responses/{kvknummer}.json', 'r', encoding='utf-8') as f:
            response = json.load(f)

        kvknummer = response.get('kvknummer')

        # Overige data
        overige_data.append({
            'kvknummer': kvknummer,
            'naam': response.get('naam'),
            'rechtsvorm': response.get('rechtsvorm'),
            'adres': response.get('adres', {}).get('straat', '') + " " + response.get('adres', {}).get('huisnummer', ''),
            'actiefsinds': response.get('actiefsinds'),
            'toelatingsvergunning': response.get('toelatingsvergunning'),
            'vestigingen': len(response.get('vestigingen', [])),
            'sbicodes': ', '.join(response.get('sbicodes', [])),
        })

        # SBI-codes op hoofdniveau
        sbi_data.append({
            'kvknummer': kvknummer,
            'sbi_codes': ', '.join(response.get('sbicodes', []))
        })




        # Functie om het huisnummer uit huisnr-toev te extraheren
        def extract_house_number(house_number_with_suffix):
            try:
                # Zoek naar het eerste gedeelte van de string dat alleen cijfers bevat
                match = re.match(r'^\d+', house_number_with_suffix)
                if match:
                    house_number = match.group(0)  # Geef de cijfers terug
                else:
                    house_number = ''

                # Zoek naar de rest van de tekens na de cijfers
                remainder = house_number_with_suffix[len(house_number):]  # Het gedeelte na de cijfers
                remainder_cleaned = re.sub(r'[^a-zA-Z0-9]', '', remainder)  # Verwijder niet-alfanumerieke tekens
                return house_number, remainder_cleaned  # Geef zowel het huisnummer als de rest terug
            except Exception as e:
                print(f"Fout bij het extraheren van het huisnummer: {e}")
                return '', ''  # Geef lege strings terug bij een fout

        # Functie om de API-aanroep te doen en de gewenste velden te extraheren

        # Functie om de PDOK API-aanroep te doen en de gewenste velden te extraheren
        def get_location_info(straat, huisnr, plaats, postcode):
            start_time = time.time()  # Starttijd loggen



            # Stel de query samen
            query = f"{straat} {huisnr} {plaats} {postcode}"

            # URL voor de PDOK API-aanroep
            url = f"https://api.pdok.nl/bzk/locatieserver/search/v3_1/suggest?q={query}&start=0&rows=1&sort=score%20desc,sortering%20asc,weergavenaam%20asc&fq=type:(gemeente%20OR%20woonplaats%20OR%20weg%20OR%20adres)&fq=bron:BAG&fl=*"

            # Headers voor de API-aanroep
            headers = {
                'Accept': 'application/json, text/plain, */*',
                'Accept-Language': 'nl,en;q=0.9,en-GB;q=0.8,en-US;q=0.7,fr;q=0.6',
                'Connection': 'keep-alive',
                'Origin': 'https://bagviewer.kadaster.nl',
                'Referer': 'https://bagviewer.kadaster.nl/',
                'Sec-Fetch-Dest': 'empty',
                'Sec-Fetch-Mode': 'cors',
                'Sec-Fetch-Site': 'cross-site',
                'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/133.0.0.0 Safari/537.36 Edg/133.0.0.0',
                'sec-ch-ua': '"Not(A:Brand";v="99", "Microsoft Edge";v="133", "Chromium";v="133"',
                'sec-ch-ua-mobile': '?0',
                'sec-ch-ua-platform': '"Windows"',
                'X-Api-Key': 'l79a392ea8073b42ae88eb255936af91b9'
            }

            try:
                # PDOK API-aanroep
                response = requests.get(url, headers=headers, timeout=1)

                if response.status_code == 200:
                    data = response.json()
                    # Check of er documenten in de response staan
                    if "response" in data and "docs" in data["response"] and len(data["response"]["docs"]) > 0:
                        doc = data["response"]["docs"][0]  # Assuming only one result
                        location_info = {
                            'adresseerbaarobject_id': doc.get('adresseerbaarobject_id', ''),
                            'geometrie_ll': doc.get('geometrie_ll', ''),
                            'weergavenaam': doc.get('weergavenaam', ''),
                            'buurtcode': doc.get('buurtcode','')
                        }

                        # Extraheren van de longitude en latitude uit geometrie_ll (POINT(lon lat))
                        geometrie_ll = location_info.get('geometrie_ll', '')
                        if geometrie_ll.startswith("POINT(") and geometrie_ll.endswith(")"):
                            coords = geometrie_ll[6:-1].split()  # Verwijder 'POINT(' en ')', split op spatie
                            if len(coords) == 2:
                                location_info['longitude'] = coords[0]
                                location_info['latitude'] = coords[1]
                            else:
                                location_info['longitude'] = None
                                location_info['latitude'] = None
                        else:
                            location_info['longitude'] = None
                            location_info['latitude'] = None
                        # API-aanroep
                        end_time = time.time()  # Eindtijd loggen
                        print(f"get_location_info voor {straat} {huisnr} duurde {end_time - start_time} seconden.")

                        return location_info
                    else:
                        print(f"Geen locatie-informatie gevonden voor adres {query}.")
                        return None
                else:
                    print(f"Fout bij het ophalen van locatiegegevens voor adres {query}: {response.status_code}")
                    return None
            except requests.exceptions.RequestException as e:
                print(f"Netwerkfout bij het ophalen van locatiegegevens voor adres {query}: {e}")
                return None

        # Functie om de BAG Kadaster API-aanroep te doen met de adresseerbaarobject_id
        def get_bag_kadaster_info(adresseerbaarobject_id):
            start_time = time.time()  # Starttijd loggen
            url = f"https://api.bag.kadaster.nl/lvbag/individuelebevragingen/v2/adresseerbareobjecten/{adresseerbaarobject_id}?expand=maaktDeelUitVan&huidig=false"

            headers = {
                'Accept-Crs': 'epsg:28992',
                'Accept': 'application/hal+json',
                'X-Api-Key': 'l79a392ea8073b42ae88eb255936af91b9'
            }

            try:
                response = requests.get(url, headers=headers, timeout=1)

                if response.status_code == 200:
                    end_time = time.time()  # Eindtijd loggen
                    print(f"get_bag_kadaster_info voor {adresseerbaarobject_id} duurde {end_time - start_time} seconden.")
                    return response.json()
                else:
                    print(f"Fout bij het ophalen van BAG Kadaster gegevens voor ID {adresseerbaarobject_id}: {response.status_code}")
                    return None
            except requests.exceptions.RequestException as e:
                print(f"Netwerkfout bij het ophalen van BAG Kadaster gegevens voor ID {adresseerbaarobject_id}: {e}")
                return None

        counter = 0
        nVestigingenInBatch = len ( response.get('vestigingen', []))
        for vestiging in response.get('vestigingen', []):  # Itereer door de vestigingen
                straat = vestiging.get('adres', {}).get('straat', '')
                huisnr = vestiging.get('adres', {}).get('huisnummer', '')
                plaats = vestiging.get('adres', {}).get('plaats', '')
                postcode = vestiging.get('adres', {}).get('postcode', '')

                # Verkrijg locatie-informatie via de PDOK API
                location_info = get_location_info(straat, huisnr, plaats, postcode)

                print(f"Count {counter} van {nVestigingenInBatch}, kvk {kvknummer}, vestiging {straat} {huisnr} {plaats}")
                counter +=1

                if location_info:
                    adresseerbaarobject_id = location_info.get('adresseerbaarobject_id', '')

                    # Verkrijg BAG Kadaster informatie met de adresseerbaarobject_id
                    if adresseerbaarobject_id:
                        bag_kadaster_info = get_bag_kadaster_info(adresseerbaarobject_id)

                        # Extract the relevant information from the BAG Kadaster response
                        if bag_kadaster_info:
                            verblijfsobject = bag_kadaster_info.get('verblijfsobject', {})
                            oppervlakte = verblijfsobject.get('verblijfsobject', {}).get('oppervlakte', None)

                            # Extract the relevant 'maaktDeelUitVan' (Pand) information
                            maakt_deel_uit_van = verblijfsobject.get('_embedded', {}).get('maaktDeelUitVan', [])
                            for pand in maakt_deel_uit_van:
                                pand_data.append({
                                    'adresseerbaarobject_id': adresseerbaarobject_id,
                                    'identificatie': pand.get('pand', {}).get('identificatie', ''),
                                    'oorspronkelijkBouwjaar': pand.get('pand', {}).get('oorspronkelijkBouwjaar', '')
                                })

                    else:
                        bag_kadaster_info = None

                    # Voeg de locatie-informatie en de BAG Kadaster gegevens toe aan de vestigingsdata
                    vestigingen_data.append({
                        'kvknummer': kvknummer,
                        'naam': vestiging.get('naam', ''),
                        'vestigingsnummer': vestiging.get('vestigingsnummer', ''),
                        'isHoofdvestiging': vestiging.get('isHoofdvestiging', False),
                        'adres': straat,
                        'huisnr-toev': huisnr,
                        'huisnr': extract_house_number(huisnr)[0],
                        'huisnr_remainder': extract_house_number(huisnr)[1],
                        'plaats': plaats,
                        'postcode': postcode,
                        'sbi_codes': ', '.join(vestiging.get('sbicodes', [])),
                        'adresseerbaarobject_id': location_info.get('adresseerbaarobject_id', ''),
                        'geometrie_ll': location_info.get('geometrie_ll', ''),
                        'weergavenaam': location_info.get('weergavenaam', ''),
                        'longitude': location_info.get('longitude', None),  # Voeg longitude toe
                        'latitude': location_info.get('latitude', None),   # Voeg latitude toe
                        'buurtcode': location_info.get('buurtcode', None),
                        'oppervlakte': oppervlakte
                    })
                else:
                    # Voeg lege velden toe als de API-aanroep faalt
                    vestigingen_data.append({
                        'kvknummer': kvknummer,
                        'naam': vestiging.get('naam', ''),
                        'vestigingsnummer': vestiging.get('vestigingsnummer', ''),
                        'isHoofdvestiging': vestiging.get('isHoofdvestiging', False),
                        'adres': straat,
                        'huisnr-toev': huisnr,
                        'huisnr': extract_house_number(huisnr)[0],
                        'huisnr_remainder': extract_house_number(huisnr)[1],
                        'plaats': plaats,
                        'postcode': postcode,
                        'sbi_codes': ', '.join(vestiging.get('sbicodes', [])),
                        'adresseerbaarobject_id': None,
                        'geometrie_ll': None,
                        'weergavenaam': None,
                        'longitude': None,
                        'latitude': None,
                        'buurtcode': None,
                        'oppervlakte': None
                    })

                # time.sleep(1)  # Wacht 1 seconde tussen de API-aanroepen om de server niet te overbelasten


    except FileNotFoundError:
        print(f"Bestand voor KvK-nummer {kvknummer} niet gevonden.")
    except json.JSONDecodeError:
        print(f"Fout bij het decoderen van JSON voor KvK-nummer {kvknummer}.")


# batch processing kvk nummers naar locatie, bagid en baginfo

In [ ]:
import os
import pandas as pd
import time
from openpyxl import load_workbook

# Functie om de laatste verwerkte batch op te slaan
def save_last_processed_batch(batch_count):
    with open('last_processed_batch.txt', 'w') as f:
        f.write(str(batch_count))

# Functie om de laatste verwerkte batch te lezen
def get_last_processed_batch():
    if os.path.exists('last_processed_batch.txt'):
        with open('last_processed_batch.txt', 'r') as f:
            return int(f.read().strip())
    else:
        return -1  # Als het bestand niet bestaat, beginnen we vanaf de eerste batch

# Functie om de eerste lege rij in een sheet te vinden
def find_first_empty_row(sheet):
    # Het aantal rijen in de sheet
    max_row = sheet.max_row
    # Als de laatste rij leeg is, dan is dat de eerste lege rij, anders de volgende
    return max_row + 1

# Functie om batches te verwerken en de resultaten toe te voegen aan Excel
def process_in_batches(kvk_nummers, batch_size=10):
    # Bepaal de laatste verwerkte batch
    last_processed_batch = get_last_processed_batch()
    print(f"Laatste verwerkte batch: {last_processed_batch}")

    # Verwerk de kvk_nummers in batches van batch_size
    batch_count = last_processed_batch + 1

    # Bepaal of het Excel-bestand al bestaat
    file_exists = os.path.exists('zorgaanbieders_data.xlsx')

    # Start een Excel-writer object aan
    with pd.ExcelWriter('zorgaanbieders_data.xlsx', engine='openpyxl', mode='a' if file_exists else 'w') as writer:
        for i in range(last_processed_batch + 1, len(kvk_nummers), batch_size):
            batch = kvk_nummers[i:i + batch_size]

            print(f"Verwerken van batch {batch_count}: KvK-nummers {batch[0]} tot {batch[-1]}")

            # Verwerk elke KvK-nummer in de huidige batch
            for kvknummer in batch:
                process_data_from_file(kvknummer)

            # Zet de gegevens om in DataFrames
            df_sbi = pd.DataFrame(sbi_data)
            df_vestigingen = pd.DataFrame(vestigingen_data)
            df_overige = pd.DataFrame(overige_data)
            df_pand = pd.DataFrame(pand_data)

            # Laad het bestaande Excel-bestand om de eerste lege rij te vinden
            if file_exists:
                # Open the workbook and get the sheet names
                workbook = load_workbook('zorgaanbieders_data.xlsx')

                # Zoek de eerste lege rij per sheet
                sheet_sbi = workbook['SBI Codes']
                sheet_vestigingen = workbook['Vestigingen']
                sheet_overige = workbook['Overige Data']
                sheet_pand = workbook['Pand Data']

                # Vind de eerste lege rij in elke sheet
                startrow_sbi = find_first_empty_row(sheet_sbi)
                startrow_vestigingen = find_first_empty_row(sheet_vestigingen)
                startrow_overige = find_first_empty_row(sheet_overige)
                startrow_pand = find_first_empty_row(sheet_pand)

                # Voeg de data van de huidige batch toe aan het Excel-bestand
                df_sbi.to_excel(writer, sheet_name='SBI Codes', index=False, header=False, startrow=startrow_sbi)
                df_vestigingen.to_excel(writer, sheet_name='Vestigingen', index=False, header=False, startrow=startrow_vestigingen)
                df_overige.to_excel(writer, sheet_name='Overige Data', index=False, header=False, startrow=startrow_overige)
                df_pand.to_excel(writer, sheet_name='Pand Data', index=False, header=False, startrow=startrow_pand)

            else:  # Eerste keer dat het bestand wordt aangemaakt, voeg de kolomhoofden toe
                df_sbi.to_excel(writer, sheet_name='SBI Codes', index=False)
                df_vestigingen.to_excel(writer, sheet_name='Vestigingen', index=False)
                df_overige.to_excel(writer, sheet_name='Overige Data', index=False)
                df_pand.to_excel(writer, sheet_name='Pand Data', index=False)

            # Log de toevoeging van de batch
            print(f"Batch {batch_count} toegevoegd aan Excel.")

            # Leeg de lijsten voor de volgende batch
            sbi_data.clear()
            vestigingen_data.clear()
            overige_data.clear()
            pand_data.clear()

            # Sla de laatste verwerkte batch op
            save_last_processed_batch(batch_count)

            # Sleep 1 seconde tussen batches om de server niet te overbelasten
            # time.sleep(1)

            # Verhoog de batch teller
            batch_count += 1

    print("Alle batches succesvol opgeslagen in 'zorgaanbieders_data.xlsx'.")




In [ ]:

# Verwerk de kvk_nummers in batches van 10
process_in_batches(kvk_nummers[:2], batch_size=1)  # Je kunt de slice wijzigen naar kvk_nummers voor volledige lijst


# take 2 batch processing

In [ ]:
import pandas as pd
import time
import os

# Functie om de laatste verwerkte batch op te slaan
def save_last_processed_batch(batch_count):
    with open('last_processed_batch.txt', 'w') as f:
        f.write(str(batch_count))

# Functie om de laatste verwerkte batch te lezen
def get_last_processed_batch():
    if os.path.exists('last_processed_batch.txt'):
        with open('last_processed_batch.txt', 'r') as f:
            return int(f.read().strip())
    else:
        return -1  # Als het bestand niet bestaat, beginnen we vanaf de eerste batch

# Functie om de tussenresultaten naar CSV op te slaan
def save_batch_to_csv(batch_count):
    # Zet de data om in DataFrame
    df_sbi = pd.DataFrame(sbi_data)
    df_vestigingen = pd.DataFrame(vestigingen_data)
    df_overige = pd.DataFrame(overige_data)
    df_pand = pd.DataFrame(pand_data)

    # Sla elke DataFrame op als CSV (appendmodus, zodat de gegevens niet overschreven worden)
    df_sbi.to_csv('sbi_codes.csv', mode='a', header=not os.path.exists('sbi_codes.csv'), index=False)
    df_vestigingen.to_csv('vestigingen.csv', mode='a', header=not os.path.exists('vestigingen.csv'), index=False)
    df_overige.to_csv('overige_data.csv', mode='a', header=not os.path.exists('overige_data.csv'), index=False)
    df_pand.to_csv('pand_data.csv', mode='a', header=not os.path.exists('pand_data.csv'), index=False)

    print(f"Batch {batch_count} opgeslagen als CSV.")

# Functie om batches te verwerken en de resultaten toe te voegen aan CSV
def process_in_batches(kvk_nummers, batch_size=10):
    last_processed_batch = get_last_processed_batch()
    print(f"Laatste verwerkte batch: {last_processed_batch}")

    # Verwerk de kvk_nummers in batches van batch_size
    batch_count = last_processed_batch + 1

    for i in range(last_processed_batch + 1, len(kvk_nummers), batch_size):
        batch = kvk_nummers[i:i + batch_size]

        print(f"Verwerken van batch {batch_count}: KvK-nummers {batch[0]} tot {batch[-1]}")


        # Verwerk elke KvK-nummer in de huidige batch
        for kvknummer in batch:
            process_data_from_file(kvknummer)

        # Sla de gegevens van deze batch op in CSV
        save_batch_to_csv( batch_count)

        # Leeg de data na het opslaan
        sbi_data.clear()
        vestigingen_data.clear()
        overige_data.clear()
        pand_data.clear()

        # Sla de laatste verwerkte batch op
        save_last_processed_batch(batch_count)

        # Sleep 1 seconde tussen batches om de server niet te overbelasten
        # time.sleep(1)

        batch_count += 1

    # Na de batchverwerking kun je de CSV-bestanden eventueel samenvoegen in een Excel-bestand
    # merge_csv_to_excel()

    print("Alle batches succesvol opgeslagen in CSV-bestanden.")

# Functie om alle CSV-bestanden samen te voegen naar Excel
def merge_csv_to_excel():
    # Laad de CSV-bestanden in DataFrames
    df_sbi = pd.read_csv('sbi_codes.csv')
    df_vestigingen = pd.read_csv('vestigingen.csv')
    df_overige = pd.read_csv('overige_data.csv')
    df_pand = pd.read_csv('pand_data.csv')

    # Zet alles in één Excel-bestand
    with pd.ExcelWriter('zorgaanbieders_data.xlsx') as writer:
        df_sbi.to_excel(writer, sheet_name='SBI Codes', index=False)
        df_vestigingen.to_excel(writer, sheet_name='Vestigingen', index=False)
        df_overige.to_excel(writer, sheet_name='Overige Data', index=False)
        df_pand.to_excel(writer, sheet_name='Pand Data', index=False)

    print("CSV-bestanden succesvol samengevoegd in 'zorgaanbieders_data.xlsx'.")




## batch proces II in Actie

In [ ]:

process_in_batches(kvk_nummers, batch_size=1)


## voeg buurcodes toe aan vestigingen.csv

In [ ]:
import pandas as pd

# Laad de eerste dataset als pure strings
df1 = pd.read_csv("vestigingen.csv", dtype=str, keep_default_na=False)

# Laad de tweede dataset als pure strings
df2 = pd.read_csv("C:/Users/DirkSackman/OneDrive - Draaijer Group SSC/Documenten - Data Experience Center/Projecten/Algemene data/2023-cbs/pc6hnr2.csv", dtype=str, keep_default_na=False)

# Maak de samengestelde sleutel in de eerste dataset
df1["PC6_Huisnummer"] = df1["postcode"].str.replace(" ", "", regex=True) + df1["huisnr"]

# Koppel de datasets op basis van PC6_Huisnummer en neem alleen Buurt2023 over
df_merged = df1.merge(df2[["PC6_Huisnummer", "Buurt2023"]], on="PC6_Huisnummer", how="left")

df_merged["Buurt2023"] = df_merged["Buurt2023"].apply(lambda x: f"BU{x}" if x else "")

# Opslaan of verder verwerken
df_merged.to_csv("vestigingen_merged.csv", index=False)

# Toon de eerste paar rijen van de gekoppelde dataset
print(df_merged.head())


# sbi codes en agb codes per vestiging

In [ ]:
import requests
import pandas as pd
import time
import os

import os
os.chdir('/content/geojsons')  # Ga naar de repo-map
# Bestanden
input_file = "vestigingen.csv"  # CSV met kvknummer en vestigingsnummer
sbi_output_file = "sbi_output.csv"
agb_output_file = "agb_output.csv"
batch_file = "last_batch.txt"

batch_size = 100  # Aantal vestigingen per batch

# Inlezen CSV als tekstvelden
df = pd.read_csv(input_file, dtype=str)

# Controleer of verplichte kolommen aanwezig zijn
if "kvknummer" not in df.columns or "vestigingsnummer" not in df.columns:
    raise ValueError("CSV-bestand moet de kolommen 'kvknummer' en 'vestigingsnummer' bevatten.")

total_records = len(df)  # Totaal aantal vestigingen

# Bepaal waar het script moet hervatten
if os.path.exists(batch_file):
    with open(batch_file, "r") as f:
        last_batch = int(f.read().strip())  # Laatste batchnummer ophalen
else:
    last_batch = 0  # Start vanaf het begin

# API-aanroep instellen
base_url = "https://zoeken.zorgaanbiedersportaal.nl/api/vestiging/"
headers = {
    'Accept': 'application/json, text/plain, */*',
    'Accept-Language': 'nl-NL,nl;q=0.9,en-US;q=0.8,en;q=0.7',
    'Connection': 'keep-alive',
    'DNT': '1',
    'Sec-Fetch-Dest': 'empty',
    'Sec-Fetch-Mode': 'cors',
    'Sec-Fetch-Site': 'same-origin',
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/133.0.0.0 Safari/537.36',
    'sec-ch-ua': '"Not(A:Brand";v="99", "Google Chrome";v="133", "Chromium";v="133"',
    'sec-ch-ua-mobile': '?0',
    'sec-ch-ua-platform': '"Windows"'
}

# Loop door batches van 100 vestigingen
for batch_start in range(last_batch * batch_size, total_records, batch_size):
    batch_end = min(batch_start + batch_size, total_records)  # Zorg dat we niet buiten de dataset gaan
    batch_number = batch_start // batch_size + 1  # Huidig batchnummer

    sbi_results = []
    agb_results = []

    for record_index, (index, row) in enumerate(df.iloc[batch_start:batch_end].iterrows(), start=1):
        kvknummer = str(row["kvknummer"]).strip()
        vestigingsnummer = str(row["vestigingsnummer"]).strip()

        url = f"{base_url}{kvknummer}/{vestigingsnummer}"
        start_time = time.time()  # Starttijd meten

        try:
            response = requests.get(url, headers=headers, timeout=20)


            if response.status_code == 429:
                print(f"🚨 429 Too Many Requests - API-limiet bereikt. Script wordt gestopt.")
                exit(1)  # Beëindig het script direct

            response.raise_for_status()
            duration = time.time() - start_time  # Duur van de API-call

            data = response.json()
            sbicodes = data.get("sbicodes", [])
            agbregistraties = data.get("agbregistraties", [])

            # Voeg SBI-codes toe
            for sbi in sbicodes:
                sbi_results.append({
                    "kvknummer": kvknummer,
                    "vestigingsnummer": vestigingsnummer,
                    "sbicode": sbi
                })

            # Voeg AGB-codes toe
            for agb in agbregistraties:
                agb_results.append({
                    "kvknummer": kvknummer,
                    "vestigingsnummer": vestigingsnummer,
                    "agbregistratie": agb
                })

            print(f"[Batch {batch_number}, Record {record_index}/100] Succes: {kvknummer} - {vestigingsnummer} (Duur: {round(duration, 3)}s)")

        except requests.exceptions.RequestException as e:
            duration = time.time() - start_time
            print(f"[Batch {batch_number}, Record {record_index}/100] Fout bij {kvknummer} - {vestigingsnummer}: {e} (Duur: {round(duration, 3)}s)")

        time.sleep(8)



    # Opslaan van batchresultaten
    sbi_df = pd.DataFrame(sbi_results)
    agb_df = pd.DataFrame(agb_results)

    # Append resultaten naar de output-bestanden
    if not sbi_df.empty:
        sbi_df.to_csv(sbi_output_file, mode='a', header=not os.path.exists(sbi_output_file), index=False)

    if not agb_df.empty:
        agb_df.to_csv(agb_output_file, mode='a', header=not os.path.exists(agb_output_file), index=False)

    # Sla het batchnummer op
    with open(batch_file, "w") as f:
        f.write(str(batch_number))

    print(f"Batch {batch_number} opgeslagen ({batch_start + 1} t/m {batch_end}).")

print("✅ Alle batches verwerkt!")


[Batch 9, Record 1/100] Succes: 04084443 - 000048315486 (Duur: 0.601s)
[Batch 9, Record 2/100] Succes: 04084443 - 000023725427 (Duur: 0.609s)
[Batch 9, Record 3/100] Succes: 04084443 - 000048314110 (Duur: 0.588s)
[Batch 9, Record 4/100] Succes: 04084443 - 000048310727 (Duur: 0.547s)
[Batch 9, Record 5/100] Succes: 04084443 - 000023725435 (Duur: 0.634s)
[Batch 9, Record 6/100] Succes: 04084443 - 000023725710 (Duur: 0.633s)
[Batch 9, Record 7/100] Succes: 04084443 - 000048310395 (Duur: 0.592s)
[Batch 9, Record 8/100] Succes: 04084443 - 000048310409 (Duur: 0.707s)
[Batch 9, Record 9/100] Succes: 04084443 - 000048310417 (Duur: 0.715s)
[Batch 9, Record 10/100] Succes: 04084443 - 000048310425 (Duur: 0.53s)
[Batch 9, Record 11/100] Succes: 04084443 - 000048310433 (Duur: 0.619s)
[Batch 9, Record 12/100] Succes: 04084443 - 000048311391 (Duur: 0.98s)
[Batch 9, Record 13/100] Succes: 04084443 - 000048311405 (Duur: 0.655s)
[Batch 9, Record 14/100] Fout bij 04084443 - 000048311413: 404 Client Error

# transform nl coords to lat lon

In [ ]:
# Installeer pyproj als je dat nog niet hebt:
# pip install pyproj

from pyproj import Transformer

# Maak een transformer object: altijd de X (longitude) als eerste parameter
transformer = Transformer.from_crs("EPSG:28992", "EPSG:4326", always_xy=True)

# Voorbeeldcoördinaten (X, Y, Z). De Z waarde wordt hier genegeerd.
x = 121972.0
y = 487360.0

# Transformeer de coördinaten
lon, lat = transformer.transform(x, y)

print("Latitude:", lat)
print("Longitude:", lon)
